In [4]:
import os
import rasterio
import geopandas as gpd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE
from rasterio.plot import show
from rasterio.features import geometry_mask

# Function to normalize bands
def normalize_band(band):
    return band / np.max(band)

# Function to calculate NDVI
def calculate_ndvi(red_band, nir_band):
    ndvi = (nir_band - red_band) / (nir_band + red_band)
    return ndvi

# Paths to Sentinel-1 and Sentinel-2 directories (replace with your paths)
sentinel1_dir = r"C:\Users\rishi\OneDrive\Desktop\sentinel_1"
sentinel2_dir = r"C:\Users\rishi\OneDrive\Desktop\sentinel_2"
shapefile_path = r"C:\Users\rishi\OneDrive\Desktop\crop_data_shapefile\merged_crop_data.shp"

# List all Sentinel-1 and Sentinel-2 files
sentinel1_files = [os.path.join(sentinel1_dir, f) for f in os.listdir(sentinel1_dir) if f.endswith('.tif')]
sentinel2_files = [os.path.join(sentinel2_dir, f) for f in os.listdir(sentinel2_dir) if f.endswith('.tif')]

# Load the shapefile using GeoPandas
gdf = gpd.read_file(shapefile_path)

# Extract features from each polygon in the shapefile
patch_size = 64  # Increased the size of patches to extract
features = []
labels = []

for sentinel1_path, sentinel2_path in zip(sentinel1_files, sentinel2_files):
    # Open and read Sentinel-1 data (assuming multi-band)
    with rasterio.open(sentinel1_path) as src:
        sentinel1_data = src.read()  # Read all bands
        sentinel1_meta = src.meta

    # Open and read Sentinel-2 data (all bands)
    with rasterio.open(sentinel2_path) as src:
        sentinel2_data = src.read()  # Read all bands
        sentinel2_meta = src.meta

    # Ensure the coordinate reference systems match
    gdf = gdf.to_crs(sentinel1_meta['crs'])

    # Extract individual bands from Sentinel-2 data
    sentinel2_red = sentinel2_data[3].astype(float)  # Red band (Band 4)
    sentinel2_nir = sentinel2_data[7].astype(float)  # Near Infrared band (Band 8)

    # Normalize Sentinel-2 bands
    sentinel2_red_norm = normalize_band(sentinel2_red)
    sentinel2_nir_norm = normalize_band(sentinel2_nir)

    # Calculate NDVI using Sentinel-2 bands
    ndvi = calculate_ndvi(sentinel2_red_norm, sentinel2_nir_norm)

    for idx, row in gdf.iterrows():
        geom = row['geometry']
        label = row['layer']  # Replace with the actual column name for crop types

        # Create a mask for the polygon
        mask = geometry_mask([geom], transform=sentinel1_meta['transform'], invert=True, out_shape=(sentinel1_meta['height'], sentinel1_meta['width']))

        # Extract patches of Sentinel-1 and Sentinel-2 data
        for i in range(0, sentinel1_meta['height'], patch_size):
            for j in range(0, sentinel1_meta['width'], patch_size):
                if mask[i:i+patch_size, j:j+patch_size].sum() > 0:  # Ensure there is some data in the patch
                    sentinel1_patch = sentinel1_data[:, i:i+patch_size, j:j+patch_size]
                    sentinel2_patch = np.array([sentinel2_red[i:i+patch_size, j:j+patch_size], sentinel2_nir[i:i+patch_size, j:j+patch_size]])
                    ndvi_patch = ndvi[i:i+patch_size, j:j+patch_size]

                    if sentinel1_patch.shape[1] == patch_size and sentinel1_patch.shape[2] == patch_size:
                        combined_patch = np.concatenate((sentinel1_patch, sentinel2_patch, np.expand_dims(ndvi_patch, axis=0)), axis=0)
                        features.append(combined_patch)
                        labels.append(label)

# Convert lists to numpy arrays
features = np.array(features)
labels = np.array(labels)

# Encode labels to integers
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Standardize features
scaler = StandardScaler()
for i in range(features.shape[1]):
    features[:, i, :, :] = scaler.fit_transform(features[:, i, :, :].reshape(-1, features[:, i, :, :].shape[-1])).reshape(features[:, i, :, :].shape)

# Handle class imbalance with SMOTE
original_shape = features.shape
features_flat = features.reshape(features.shape[0], -1)
smote = SMOTE(random_state=42)
features_resampled, labels_resampled = smote.fit_resample(features_flat, labels_encoded)
features_resampled = features_resampled.reshape(-1, original_shape[1], original_shape[2], original_shape[3])

# Initialize RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_resampled, labels_resampled, test_size=0.2, random_state=42)

# Flatten X_train and X_test for RandomForestClassifier
X_train_flatten = X_train.reshape(X_train.shape[0], -1)
X_test_flatten = X_test.reshape(X_test.shape[0], -1)

# Train the RandomForestClassifier
rf_classifier.fit(X_train_flatten, y_train)

# Predict on the test set
y_pred = rf_classifier.predict(X_test_flatten)

# Print classification report
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


                           precision    recall  f1-score   support

Updated_Crop_class_Garlic       0.57      0.65      0.61        43
Updated_Crop_class_Onion1       0.36      0.50      0.42        34
Updated_Crop_class_Wheat1       0.75      0.66      0.70        50
Updated_Crop_class_Wheat2       0.59      0.40      0.48        40

                 accuracy                           0.56       167
                macro avg       0.57      0.55      0.55       167
             weighted avg       0.59      0.56      0.57       167



In [1]:
import os
import rasterio
import geopandas as gpd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE
from rasterio.plot import show
from rasterio.features import geometry_mask
import joblib

# Function to normalize bands
def normalize_band(band):
    return band / np.max(band)

# Function to calculate NDVI
def calculate_ndvi(red_band, nir_band):
    ndvi = (nir_band - red_band) / (nir_band + red_band)
    return ndvi

# Paths to Sentinel-1 and Sentinel-2 directories (replace with your paths)
sentinel1_dir = r"C:\Users\rishi\OneDrive\Desktop\sentinel_1"
sentinel2_dir = r"C:\Users\rishi\OneDrive\Desktop\sentinel_2"
shapefile_path = r"C:\Users\rishi\OneDrive\Desktop\crop_data_shapefile\merged_crop_data.shp"

# List all Sentinel-1 and Sentinel-2 files
sentinel1_files = [os.path.join(sentinel1_dir, f) for f in os.listdir(sentinel1_dir) if f.endswith('.tif')]
sentinel2_files = [os.path.join(sentinel2_dir, f) for f in os.listdir(sentinel2_dir) if f.endswith('.tif')]

# Load the shapefile using GeoPandas
gdf = gpd.read_file(shapefile_path)

# Extract features from each polygon in the shapefile
patch_size = 64  # Increased the size of patches to extract
features = []
labels = []

for sentinel1_path, sentinel2_path in zip(sentinel1_files, sentinel2_files):
    # Open and read Sentinel-1 data (assuming multi-band)
    with rasterio.open(sentinel1_path) as src:
        sentinel1_data = src.read()  # Read all bands
        sentinel1_meta = src.meta

    # Open and read Sentinel-2 data (all bands)
    with rasterio.open(sentinel2_path) as src:
        sentinel2_data = src.read()  # Read all bands
        sentinel2_meta = src.meta

    # Ensure the coordinate reference systems match
    gdf = gdf.to_crs(sentinel1_meta['crs'])

    # Extract individual bands from Sentinel-2 data
    sentinel2_red = sentinel2_data[3].astype(float)  # Red band (Band 4)
    sentinel2_nir = sentinel2_data[7].astype(float)  # Near Infrared band (Band 8)

    # Normalize Sentinel-2 bands
    sentinel2_red_norm = normalize_band(sentinel2_red)
    sentinel2_nir_norm = normalize_band(sentinel2_nir)

    # Calculate NDVI using Sentinel-2 bands
    ndvi = calculate_ndvi(sentinel2_red_norm, sentinel2_nir_norm)

    for idx, row in gdf.iterrows():
        geom = row['geometry']
        label = row['layer']  # Replace with the actual column name for crop types

        # Create a mask for the polygon
        mask = geometry_mask([geom], transform=sentinel1_meta['transform'], invert=True, out_shape=(sentinel1_meta['height'], sentinel1_meta['width']))

        # Extract patches of Sentinel-1 and Sentinel-2 data
        for i in range(0, sentinel1_meta['height'], patch_size):
            for j in range(0, sentinel1_meta['width'], patch_size):
                if mask[i:i+patch_size, j:j+patch_size].sum() > 0:  # Ensure there is some data in the patch
                    sentinel1_patch = sentinel1_data[:, i:i+patch_size, j:j+patch_size]
                    sentinel2_patch = np.array([sentinel2_red[i:i+patch_size, j:j+patch_size], sentinel2_nir[i:i+patch_size, j:j+patch_size]])
                    ndvi_patch = ndvi[i:i+patch_size, j:j+patch_size]

                    if sentinel1_patch.shape[1] == patch_size and sentinel1_patch.shape[2] == patch_size:
                        combined_patch = np.concatenate((sentinel1_patch, sentinel2_patch, np.expand_dims(ndvi_patch, axis=0)), axis=0)
                        features.append(combined_patch)
                        labels.append(label)

# Convert lists to numpy arrays
features = np.array(features)
labels = np.array(labels)

# Encode labels to integers
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Standardize features
scaler = StandardScaler()
for i in range(features.shape[1]):
    features[:, i, :, :] = scaler.fit_transform(features[:, i, :, :].reshape(-1, features[:, i, :, :].shape[-1])).reshape(features[:, i, :, :].shape)

# Handle class imbalance with SMOTE
original_shape = features.shape
features_flat = features.reshape(features.shape[0], -1)
smote = SMOTE(random_state=42)
features_resampled, labels_resampled = smote.fit_resample(features_flat, labels_encoded)
features_resampled = features_resampled.reshape(-1, original_shape[1], original_shape[2], original_shape[3])

# Initialize RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_resampled, labels_resampled, test_size=0.2, random_state=42)

# Flatten X_train and X_test for RandomForestClassifier
X_train_flatten = X_train.reshape(X_train.shape[0], -1)
X_test_flatten = X_test.reshape(X_test.shape[0], -1)

# Train the RandomForestClassifier
rf_classifier.fit(X_train_flatten, y_train)

# Predict on the test set
y_pred = rf_classifier.predict(X_test_flatten)

# Print classification report
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# Save the trained model to the local device
model_path = r"C:\Users\rishi\OneDrive\Desktop\random_forest_model.joblib"
joblib.dump(rf_classifier, model_path)

print(f"Model saved to {model_path}")


                           precision    recall  f1-score   support

Updated_Crop_class_Garlic       0.57      0.65      0.61        43
Updated_Crop_class_Onion1       0.36      0.50      0.42        34
Updated_Crop_class_Wheat1       0.75      0.66      0.70        50
Updated_Crop_class_Wheat2       0.59      0.40      0.48        40

                 accuracy                           0.56       167
                macro avg       0.57      0.55      0.55       167
             weighted avg       0.59      0.56      0.57       167

Model saved to C:\Users\rishi\OneDrive\Desktop\random_forest_model.joblib
